<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/new_decision_tree_and_regr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [54]:
from IPython.display import SVG, display, HTML

In [55]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [56]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  coref_df_anns60k.csv  new_anns60k.csv	new_df_anns60k.csv  sample_data
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [57]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-19 21:23:32--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162863 (1.1M) [text/plain]
Saving to: ‘coref_df_anns60k.csv.1’

coref_df_anns60k.cs 100%[===================>]   1.11M  --.-KB/s    in 0.02s   

2021-05-19 21:23:32 (60.2 MB/s) - ‘coref_df_anns60k.csv.1’ saved [1162863/1162863]

--2021-05-19 21:23:32--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reque

In [58]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
# with open('coref_df_anns60k.csv') as csvfile:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      # print(new_row)
      if len(new_row) > 9:
        # print(new_row[:9])
        new_row = new_row[:9]
      writer.writerow(new_row)


In [60]:
# files.download('new_df_anns60k.csv')

In [61]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

In [62]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1.0,0.0,0.0,0.0,0.0,обсуждаем,0.0,0.0
1,препарат,1.0,0.0,0.0,1.0,0.0,обсуждаем,0.0,0.0
2,рисперидон,1.0,0.0,0.0,2.0,0.0,препарат,1.0,2.0
3,рисполепт,1.0,0.0,0.0,3.0,0.0,рисперидон,1.0,3.0
4,сперидан,1.0,0.0,0.0,4.0,0.0,рисполепт,1.0,4.0
...,...,...,...,...,...,...,...,...,...
15405,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15406,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15407,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15408,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [63]:
import numpy as np

In [64]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')

In [65]:

np.matrix(cor_df.HEAD_VECT[0]).A[0]

array([-2.08233938e-01,  9.96180058e-01, -2.79894114e-01, -9.09143448e-01,
        3.49961907e-01,  1.22820950e+00, -4.53999013e-01, -4.96644318e-01,
       -5.06227851e-01, -3.50297660e-01, -2.68426925e-01, -3.18647206e-01,
       -4.94759202e-01, -4.05789405e-01, -6.19022734e-02,  5.76916397e-01,
        6.45665407e-01, -6.29692733e-01, -2.28346493e-02,  4.74443585e-01,
        2.75724858e-01,  1.00444555e-01, -2.81267107e-01,  7.38310993e-01,
        1.64172634e-01, -1.40955552e-01, -1.34702528e+00, -2.28699774e-01,
       -2.00072408e-01,  1.53492257e-01,  2.15266779e-01, -3.75066400e-01,
        9.79104459e-01,  1.09640598e-01,  4.11819309e-01, -5.48374474e-01,
        5.74254513e-01,  1.78177617e-02,  2.20505193e-01,  8.06000233e-01,
       -3.45855951e-01, -9.48401261e-03, -1.23907340e+00, -7.95203969e-02,
       -7.97536790e-01, -5.43104485e-02,  4.87657279e-01,  5.39513767e-01,
       -2.17315450e-01, -3.24471802e-01, -3.16938460e-02, -4.50928695e-02,
       -3.87996256e-01,  

In [66]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

In [67]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]

In [68]:
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]

In [69]:
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]

In [70]:
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

In [71]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,"[-0.208233938, 0.996180058, -0.279894114, -0.9...",0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
1,препарат,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,"[0.5048941, 0.81449467, 0.29763442, 0.60393727...",0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,"[0.7365944, -0.48505062, -0.00257558, 0.168963...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.41243815, 0.190811, -0.11933009, 0.04242903...",NaN,NaN
26493,таможенные,"[0.73242044, -0.24206167, -0.6556247, -0.47665...",0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,"[0.7365944, -0.48505062, -0.00257558, 0.168963...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.41243815, 0.190811, -0.11933009, 0.04242903...",NaN,NaN
26494,пошлины,"[0.7365944, -0.48505062, -0.00257558, 0.168963...",0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,"[-0.0050342, 0.14796825, 0.08416066, -0.776544...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.57842094, 0.09196201, -0.19555457, -0.26701...",NaN,NaN
26495,как,"[1.0349948, 0.73793733, -0.38374668, -0.544819...",0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,"[-0.0050342, 0.14796825, 0.08416066, -0.776544...","[0.100772209, 0.0629534647, -0.035381455, -0.0...","[0.57842094, 0.09196201, -0.19555457, -0.26701...",NaN,NaN


In [72]:
df = cor_df.iloc[:8060,:]

In [73]:
len(df.loc[df['NER'] == 1])

351

In [74]:
len(df.loc[df['NER'] == 3])

69

In [75]:
len(df.loc[df['NER'].isin([1,2,3])])

544

In [76]:
len(df.loc[df['NER'] == 2])

124

In [77]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
21,лекарством,"[0.598588, -0.07832722, 0.23088205, -0.3704158...",0,1,1,1,10,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.3479745, 0.34477097, 0.20343542, 0.10220611...","[0.05498165, -0.00549405, 0.01865345, -0.03142...","[0.42120528, 0.21546456, 0.00441866, -0.240917...",2.0,0.0
142,рисполепта,"[0.20826544, 0.5118653, 0.6703367, -0.22502856...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.5131939, 0.9708054, 0.53676724, 0.09788364,...","[0.11585727, 0.1283627, 0.02299715, -0.0095005...","[0.28126505, 0.3790152, 0.5397224, -0.09368, -...",1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,лирике,"[0.12357176, 0.362139, 0.6748619, 0.35437727, ...",1,5,45,2,1713,inan,loct,femn,NaN,NOUN,obl,добавить,NaN,NaN,NaN,INFN,"[0.7259974, 1.2100255, 0.917794, 0.738014, -0....","[0.05847374, 0.04148796, 0.05273903, 0.0201881...","[0.45487276, 0.46911898, 0.68700504, 0.3107689...",1.0,1717.0
7920,лирики,"[0.0137766665, 0.403955668, -0.0423638336, -0....",1,5,45,3,1718,inan,gent,femn,NaN,NOUN,obl,сошли,NaN,NaN,NaN,VERB,"[0.4408794, 0.00925178, -0.4723476, 0.10177562...","[0.1076632, 0.10290281, -0.03424522, -0.009713...","[0.4028684, 0.4570796, -0.1217413, -0.06683407...",1.0,1722.0
7927,нее,"[1.2529131, 0.32384905, 1.1935774, 0.1080301, ...",1,5,45,4,1725,NaN,gent,femn,3per,NPRO,nmod,месяц,inan,accs,masc,NOUN,"[0.26192164, 0.52908844, 0.4179378, 0.6429006,...","[0.03176786, 0.16602686, 0.03493694, -0.009230...","[0.6401785, 0.13630565, 0.36455706, 0.10072222...",3.0,1722.0
7932,сейзар,"[0.528584778, 0.395830333, 0.227217048, -0.216...",1,5,45,5,1730,anim,gent,femn,NaN,NOUN,obj,пила,inan,nomn,femn,NOUN,"[0.578148, 0.69822013, 0.5071935, -0.04057771,...","[-0.00699353265, 0.0394528359, 0.0887183025, 0...","[0.10931073, 0.6362406, 0.37241444, 0.30618608...",1.0,1734.0


# **SEMANTIC AXIS**

In [78]:
df['TOPIC_NUM'] == 0

0       False
1       False
2       False
3       False
4       False
        ...  
8055    False
8056    False
8057    False
8058    False
8059    False
Name: TOPIC_NUM, Length: 8060, dtype: bool

In [79]:
(df['TOPIC_NUM'] == 0)
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1.0,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1.0,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1.0,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1.0,3.0


In [80]:
sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

array([ 0.11242506,  1.06504189,  0.60962812, -0.1098321 , -0.25309914,
        0.18837714, -0.29014948, -0.66894132,  0.12588727, -0.36141735,
       -0.05396164, -0.35577641, -0.14144993, -0.15987718, -0.68407827,
       -0.04969584,  0.04704007,  0.06857683,  0.14719775, -0.42784514,
        0.12565804, -0.09923978, -0.46527826,  0.02228641, -0.29075186,
        0.19580094, -0.26369618, -0.26473205, -0.52919418,  0.07394732,
       -0.1234136 , -0.1277977 , -0.02898717,  0.14748353,  0.18349706,
       -0.02373637,  0.25928119, -0.35122524, -0.0759414 ,  0.94526116,
        0.08296286,  0.64028589, -0.28902657,  0.08378753, -0.44904899,
       -0.24238156,  0.36918995, -0.20184582, -0.20304089, -0.099586  ,
        0.13669602,  0.38995985, -0.19354978, -0.37518906,  0.09664967,
        0.01878619, -0.22422224, -0.06534558,  0.09065832, -0.05079969,
       -0.09178696, -0.39066393, -0.03679946, -0.37383265, -0.24993176,
       -0.28516305,  0.65741815, -0.18487955,  0.46176331,  0.12

In [81]:
token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [82]:
df.TOKEN_VECT[710]

array([nan])

In [83]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [84]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [85]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.424228,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0.0,0.0
1,препарат,0.403455,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0.0,0.0
2,рисперидон,0.933164,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.987311,0.904469,0.917969,1.0,2.0
3,рисполепт,0.863874,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.415625,0.904469,0.856327,1.0,3.0
4,сперидан,0.806514,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.438658,0.904469,0.806874,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,400,0.410595,1,5,48,0,1853,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0.295454,0.487671,0.687963,0.0,0.0
8056,мг,0.611221,1,5,48,0,1854,inan,nomn,masc,NaN,NOUN,nmod,пп,NaN,NaN,NaN,NaN,0.266755,0.487671,0.609623,0.0,0.0
8057,ибупрофена,0.632453,1,5,48,0,1855,inan,gent,masc,NaN,NOUN,nmod,мг,inan,nomn,masc,NOUN,0.295454,0.487671,0.687963,0.0,0.0
8058,ибупрофен,0.637485,1,5,48,0,1856,NaN,NaN,masc,NaN,ADJS,obj,\n\n,NaN,NaN,NaN,NaN,0.000000,0.487671,0.725556,0.0,0.0


In [86]:
df.describe()

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
count,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000
mean,0.357982,0.942680,3.239082,18.959305,3.981390,1164.631266,0.363677,0.518268,0.459602,0.100000,66.794045
std,0.150429,0.232468,1.180049,13.343202,5.343252,1015.556727,0.102299,0.116242,0.146363,0.414916,355.810749
min,-0.044138,0.000000,1.000000,0.000000,0.000000,0.000000,-0.082418,0.257439,0.000000,0.000000,0.000000
25%,0.251422,1.000000,3.000000,7.000000,1.000000,314.000000,0.304791,0.446664,0.377064,0.000000,0.000000
50%,0.335851,1.000000,3.000000,18.000000,2.000000,899.500000,0.360753,0.499119,0.462297,0.000000,0.000000
75%,0.423863,1.000000,4.000000,29.000000,5.000000,1755.250000,0.415283,0.558765,0.547568,0.000000,0.000000
max,0.934259,1.000000,5.000000,49.000000,31.000000,3668.000000,0.987311,0.928780,0.918316,3.000000,3653.000000


In [87]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [88]:

_, lbl = pd.factorize(df.ANIMACY)
print(lbl[0])
_

inan


array([-1,  0,  1, ...,  0, -1,  0])

In [89]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'аскофен', 'ибупрофена', 'yurij76', 'называется', 'составе', 'кофеина',
        'ацетилки', 'сказывается', 'кровообращении', 'ибупрофен'],
       dtype='object', length=1576),
 'HEAD_ANIMACY': Index(['inan', 'anim'], dtype='object'),


In [90]:
# X = cor_df.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# X_test = cor_df_test.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [91]:
train, test = train_test_split(df, test_size=0.2)

In [92]:
train = train.reset_index()

In [93]:

# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
train = train.drop(['TOKEN', 'index'], axis=1)

In [94]:
test = test.reset_index()

In [95]:
test

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,3023,уже,0.282847,1,3,15,9,1139,-1,-1,-1,-1,8,5,556,-1,-1,1,0,0.338932,0.458322,0.485607,0.0,0.0
1,344,рисполепта,0.790397,1,1,2,4,64,0,0,1,-1,1,21,147,0,5,0,1,0.360753,0.823316,0.526785,1.0,344.0
2,6619,или,0.200896,1,5,19,0,417,-1,-1,-1,-1,4,13,1305,0,0,0,1,0.299040,0.415296,0.365752,0.0,0.0
3,2593,обсуждала,0.438473,1,3,8,13,709,-1,-1,1,-1,0,0,653,-1,-1,1,0,0.407874,0.565218,0.557357,0.0,0.0
4,3433,44,0.351492,1,3,18,15,1549,-1,-1,-1,-1,-1,20,663,0,1,0,1,0.330449,0.491657,0.225133,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,3799,olzapin,0.527160,1,3,23,4,1915,-1,-1,-1,-1,-1,0,765,-1,-1,-1,-1,0.529255,0.557775,0.452479,0.0,0.0
1608,7993,аскофен,0.673179,1,5,46,0,1791,0,0,0,-1,1,4,336,-1,-1,-1,-1,0.000000,0.727068,0.680814,0.0,0.0
1609,6486,людей,0.279347,1,5,13,1,284,1,1,0,-1,1,21,1265,0,1,2,1,0.461832,0.365501,0.462307,0.0,0.0
1610,583,практике,0.436726,0,2,1,0,5,0,4,1,-1,1,10,235,-1,-1,0,0,0.413990,0.512015,0.555716,0.0,0.0


In [96]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [97]:
test

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,3023,уже,0.282847,1,3,15,9,1139,-1,-1,-1,-1,8,5,556,-1,-1,1,0,0.338932,0.458322,0.485607,0.0,0.0
1,344,рисполепта,0.790397,1,1,2,4,64,0,0,1,-1,1,21,147,0,5,0,1,0.360753,0.823316,0.526785,1.0,344.0
2,6619,или,0.200896,1,5,19,0,417,-1,-1,-1,-1,4,13,1305,0,0,0,1,0.299040,0.415296,0.365752,0.0,0.0
3,2593,обсуждала,0.438473,1,3,8,13,709,-1,-1,1,-1,0,0,653,-1,-1,1,0,0.407874,0.565218,0.557357,0.0,0.0
4,3433,44,0.351492,1,3,18,15,1549,-1,-1,-1,-1,-1,20,663,0,1,0,1,0.330449,0.491657,0.225133,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,3799,olzapin,0.527160,1,3,23,4,1915,-1,-1,-1,-1,-1,0,765,-1,-1,-1,-1,0.529255,0.557775,0.452479,0.0,0.0
1608,7993,аскофен,0.673179,1,5,46,0,1791,0,0,0,-1,1,4,336,-1,-1,-1,-1,0.000000,0.727068,0.680814,0.0,0.0
1609,6486,людей,0.279347,1,5,13,1,284,1,1,0,-1,1,21,1265,0,1,2,1,0.461832,0.365501,0.462307,0.0,0.0
1610,583,практике,0.436726,0,2,1,0,5,0,4,1,-1,1,10,235,-1,-1,0,0,0.413990,0.512015,0.555716,0.0,0.0


In [98]:
X, y, X_test, y_test = train.iloc[:,:23], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.drop(['TOKEN', 'index'], axis=1).COREFERENCE_CLUSTER

In [99]:
X

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,0.375473,1,5,21,0,507,0,1,1,-1,1,12,114,-1,-1,-1,5,0.340372,0.485148,0.535091,0.0,0.0
1,0.425671,1,3,36,5,3037,-1,-1,-1,-1,8,17,695,0,1,2,1,0.381434,0.404109,0.441392,0.0,0.0
2,0.475238,1,2,13,3,695,0,1,0,-1,1,1,287,-1,-1,-1,5,0.371452,0.468809,0.513213,0.0,0.0
3,0.273668,1,3,23,15,2156,0,1,2,-1,1,10,835,-1,-1,0,6,0.528965,0.449133,0.530611,0.0,0.0
4,0.359032,1,3,8,7,669,-1,-1,-1,-1,8,5,4,-1,-1,-1,0,0.395409,0.492354,0.563909,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,0.309848,1,3,28,0,2566,-1,-1,-1,-1,8,5,972,0,1,1,1,0.379809,0.498667,0.346992,0.0,0.0
6444,0.268735,1,2,9,0,427,0,0,1,-1,1,12,376,-1,-1,2,0,0.383132,0.465443,0.332992,0.0,0.0
6445,0.152379,1,2,11,0,564,-1,-1,-1,-1,7,9,236,0,5,1,1,0.372338,0.512015,0.224044,0.0,0.0
6446,0.284003,1,3,13,7,879,-1,-1,-1,-1,-1,20,445,0,1,0,1,0.207489,0.300015,0.460425,0.0,0.0


In [100]:
y

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6443    0.0
6444    0.0
6445    0.0
6446    0.0
6447    0.0
Name: COREFERENCE_CLUSTER, Length: 6448, dtype: float64

In [165]:
clf = tree.DecisionTreeClassifier(max_depth=200)
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=200, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [166]:
clf_ent = tree.DecisionTreeClassifier(criterion='entropy', max_depth=200)
clf_ent.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=200, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [167]:
from graphviz import Source
from sklearn import tree
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [168]:
clf.score(X_test, y_test), clf_ent.score(X_test, y_test)

(0.956575682382134, 0.9578163771712159)

In [169]:
# tree.plot_tree(clf)

In [170]:
# graph = Source(tree.export_graphviz(clf, out_file=None, 
#                                     feature_names=list(X), class_names=uniques, filled=True))

# display(SVG(graph.pipe(format='svg')))

In [171]:
pred = clf.predict(X_test.drop('NER', axis=1))

ValueError: ignored

In [173]:
pred_ent = clf_ent.predict(X_test)

In [174]:
accuracy_score(list(pred), list(y_test))

0.955955334987593

In [175]:
accuracy_score(list(pred_ent), list(y_test))

0.9578163771712159

In [176]:
for i in range(len(pred)):
  if pred[i] == y_test[i] and pred[i] != 0:
    print(pred[i])
    print(y_test[i])
    print(test.iloc[:,:24].iloc[[i]])

1657.0
1657.0
     index   TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
156   7855  лирики    0.620593  ...          0.593643  1.0               1657.0

[1 rows x 24 columns]
270.0
270.0
     index TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
297   6502    он    0.329561  ...          0.530246  3.0                270.0

[1 rows x 24 columns]
270.0
270.0
     index      TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
443   6468  тиапридал    0.826306  ...          0.622012  1.0                270.0

[1 rows x 24 columns]
5.0
5.0
     index TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
624   5563    он    0.329561  ...          0.551001  3.0                  5.0

[1 rows x 24 columns]
183.0
183.0
     index      TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
865   2050  препарата    0.281452  ...          0.461852  2.0                183.0

[1 rows x 24 columns]
446.0
446.0
     index 

In [177]:
for i in range(len(pred)):
  if pred_ent[i] == y_test[i] and pred_ent[i] != 0:
    print(pred_ent[i])
    print(y_test[i])
    print(test.iloc[:,:24].iloc[[i]])

270.0
270.0
     index TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
297   6502    он    0.329561  ...          0.530246  3.0                270.0

[1 rows x 24 columns]
5.0
5.0
     index TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
624   5563    он    0.329561  ...          0.551001  3.0                  5.0

[1 rows x 24 columns]
183.0
183.0
     index      TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
865   2050  препарата    0.281452  ...          0.461852  2.0                183.0

[1 rows x 24 columns]
446.0
446.0
     index TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
942   2323   них    0.203623  ...          0.409583  3.0                446.0

[1 rows x 24 columns]
365.0
365.0
     index        TOKEN  TOKEN_VECT  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
946   5918  галоперидол    0.897747  ...          0.604788  1.0                365.0

[1 rows x 24 columns]
287.0
287.0
     index   

In [113]:
# tp = []
# fp = []
# tn = []
# fn = []
# for i in range(len(pred)):
#   if pred[i] != 0.0:
#     if pred[i] == y_test[i]:
#       t.append(i)
#     else:
#       f.append(i)

In [152]:
test.iloc[:,22:23].iloc[[1]].values[0][0] in (2.0, 2.0)

False

In [179]:
pred_ent = clf_ent.predict(X_test)
t = []
f = []

for i in range(len(pred)):
  if pred_ent[i] != 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
    if pred_ent[i] == y_test[i]:
      t.append(i)
    else:
      f.append(i)

In [180]:

len(t), len(f)

(12, 8)

In [181]:
precision_ent = len(t)/(len(f) + len(t))
precision_ent

0.6

In [118]:
from sklearn.ensemble import RandomForestClassifier

In [182]:
randf_clf = RandomForestClassifier(max_depth=15, n_estimators=600, min_samples_split=4, bootstrap=True, )
randf_clf.fit(X.drop('NER', axis=1), y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [158]:
randf_clf.score(X_test.drop('NER', axis=1), y_test)

0.9491315136476427

In [159]:
pred_randf = randf_clf.predict(X_test.drop('NER', axis=1))


t = []
f = []

for i in range(len(pred)):
  if pred_randf[i] != 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
    if pred_randf[i] == y_test[i]:
      t.append(i)
    else:
      f.append(i)
len(t), len(f)

(2, 12)

In [160]:
from sklearn import linear_model

In [161]:
reg = linear_model.LinearRegression()
reg.fit(X.drop('NER', axis=1), y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [126]:
reg.score(X_test.drop('NER', axis=1), y_test)

1.0

In [162]:
pred_reg = reg.predict(X_test.drop('NER', axis=1))


t = []
f = []

for i in range(len(pred)):
  if pred_reg[i] != 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
    if pred_reg[i] == y_test[i]:
      t.append(i)
    else:
      f.append(i)
len(t), len(f)

(1, 26)